<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
replace cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

In [ ]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

Data preprocessing

In [ ]:
data_train.head(3)

In [ ]:
data = pd.concat([data_train, data_test, data_validation ])
train_df = data

In [ ]:
train_df.shape

In [ ]:
sample=data_train.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

Checking for Null values


In [ ]:
data_train.isna().sum()

Checking for duplicates


In [ ]:
data_train.duplicated(subset=['article','highlights']).sum()

In [ ]:
data_train=data_train.drop_duplicates(subset=['article','highlights'])
data_train.shape

In [ ]:
data_train.duplicated(subset=['article','highlights']).sum()

In [ ]:
data_train['article'][:10]

In [ ]:
data_train['article'] = data_train['article'].str.lower()
data_train['highlights'] = data_train['highlights'].str.lower()

data_train.head()

In [ ]:
data_train['article']=data_train['article'].str.lower()

data_train['highlights']=data_train['highlights'].str.lower()

Converting to lower case

In [ ]:
data_train.head()

In [ ]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [ ]:
data_train['article'] = data_train['article'].apply(remove_html_tags)

data_train['highlights'] = data_train['highlights'].apply(remove_html_tags)

In [ ]:
data_train.head()

In [ ]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [ ]:
data_train['article'] = data_train['article'].apply(remove_punc)

data_train['highlights'] = data_train['highlights'].apply(remove_punc)

In [ ]:
from nltk.corpus import stopwords
stopwords=stopwords.words("english")
print(stopwords)

In [ ]:
!pip install contractions


In [ ]:
def re_clean(text):
    import re
    text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', ' ', text)
    text = re.sub(r'[_\-;%()|+&=*%:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text=re.sub(r'\n',' ',text)
    text=re.sub(' est ',' ',text)
    text=re.sub(r'[?!]','.',text)
    return text
#used to expand contractions
def expand(text):
    import contractions
    text=text.split()
    final=[]
    for word in text:
        try:
            final.append(contractions.fix(word)+" ")
        except:
            final.append(word+" ")
            print(word)
    return "".join(final)
def remove(texts):
    final=[]
    for text in texts:
        sents=[]
        sentences=text.split(".")
        for sentence in sentences:
            if(len(sentence.split())>=5):
                sents.append(sentence+".")
        final.append("".join(sents))
    return final
def removeTag(texts):
    final=[]
    #removing cnn and est
    for text in texts:
        cnn=text.find("cnn")
        if(cnn!=-1 and cnn<len(text)//10):
            text=text[cnn+3:]
        found=False
        for i in range(2):
            est=text.find(" est,")
            if(est<len(text)//5 and est!=-1):
                text=text[est+5:]
                found=True
        fs=text.find(".")
        if(fs<20 and fs!=-1 and found):
            text=text[fs:]
        final.append(text)
    return final

In [ ]:
texts=data_train.iloc[:,1]

In [ ]:
new_texts=[]
for i in texts:
    new_texts.append(expand(re_clean(i)).lower())
finalcleaned=removeTag(remove(new_texts))

In [ ]:
summaries=data_train.iloc[:,2]

In [ ]:
new_summaries=[]
for i in summaries:
    new_summaries.append(expand(re_clean(i)).lower())

In [ ]:
newdf=pd.DataFrame()
newdf["Texts"]=finalcleaned
newdf["Summaries"]=new_summaries

In [ ]:
newdf.to_csv("Cleaned.csv")

In [ ]:
newdf

In [ ]:
def words(texts):
    import re
    setofwords=set()
    for text in texts:
        text=re.sub(r'[.,]',' ',text)
        words=text.split()
        for word in words:
            setofwords.add(word)
    return setofwords
swords=words(finalcleaned)